In [1]:
import ee
import geemap
import geopandas as gpd

ee.Authenticate()

ee.Initialize()
ee.ServiceAccountCredentials(email = "ajs-gee-access@earthengineajs.iam.gserviceaccount.com", 
                              key_file="/Users/Anthony/Documents/GoogleCloud/API_Key/earthengineajs-ajs_gee_access.json")

/opt/miniconda3/envs/geospatial-processing/lib/python3.13/site-packages/geemap/conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Enter verification code:  4/1AVMBsJhCLr3eXT78XH14hLGsG5ymCEYjSDodBkiffCv-9F96wTcB-ZEUJew



Successfully saved authorization token.


In [2]:
%config IPCompleter.use_jedi = True

In [3]:
import os
from google.cloud import storage

# Set up authentication for Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/Anthony/Documents/GoogleCloud/API_Key/earthengineajs-ajs_gee_access.json"

# Test connection
client = storage.Client()
buckets = list(client.list_buckets())
print(f"Available buckets: {[bucket.name for bucket in buckets]}")

Available buckets: ['ajs_gee_data']


In [4]:
storage_client = storage.Client()
bucket_name = "ajs_gee_data"
bucket = storage_client.bucket(bucket_name)
bucket

<Bucket: ajs_gee_data>

Geometry of interest

In [5]:

gdf = gpd.read_file("/Users/Anthony/Data and Analysis Local/NYS_Wetlands_GHG/Data/NWI/NY_6350.gpkg")
gdf_huc = gpd.read_file("/Users/Anthony/Data and Analysis Local/NYS_Wetlands_GHG/Data/NY_HUCS/SaranacRiver_043001060504.gpkg")

gdf = gdf.to_crs(4326)
gdf_huc = gdf_huc.to_crs(4326)

gjson = gdf.__geo_interface__
gdf.crs
gjson_huc = gdf_huc.__geo_interface__

In [6]:
gfc = ee.FeatureCollection(gjson)
gfc_huc = ee.FeatureCollection(gjson_huc)

In [15]:
Map = geemap.Map(center=[43.23, -75.89], zoom=6)
Map.add_basemap("HYBRID")
Map.addLayer(gfc, {}, "NY State")
Map.addLayer(gfc_huc, {}, "SaranacRiverHUC")
Map

Map(center=[43.23, -75.89], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [8]:

# Define Image ImageCollection
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
naip = ee.ImageCollection("USDA/NAIP/DOQQ")

# Define time period
start_date = '2021-01-01'
end_date = '2025-08-15'

# Define start and end DOY
start_doy = 121
end_doy = 273

In [9]:
def calculate_SAR_indices(image):
    VV = image.select("VV")
    VH = image.select("VH")

    # DpRVIVV = image.expression(
    #     "(4.0 * VH) / (VV + VH)",
    #     {"VH": VH, "VV": VV}
    # ).rename("DpRVIVV")
    DpRVIVV = VV.add(VH).divide(VH.multiply(4.0)).rename("DpRVIVV")
    
    image.addBands([DpRVIVV, VV, VH])

    return image

def calculate_indices(image):
    # Extract bands
    NIR = image.select('B8')
    RED = image.select('B4')
    REDG2 = image.select("B6")
    GREEN = image.select('B3')
    BLUE = image.select('B2')
    SWIR1 = image.select('B11')

    # Calculate spectral indices
    NDVI = (NIR.subtract(RED)).divide(NIR.add(RED)).rename('NDVI')
    MNDWI = (GREEN.subtract(SWIR1)).divide(GREEN.add(SWIR1)).rename('MNDWI')
    PSRI = (RED.subtract(BLUE)).divide(REDG2).rename("PSRI") # Plant senescence 
    NDYI = (GREEN.subtract(BLUE)).divide(GREEN.add(BLUE)).rename('NDYI') #Yellow
    
    # Calculate EVI (Sentinel2)
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {'NIR': NIR, 'RED': RED, 'BLUE': BLUE}
    ).rename('EVI')


    # Add bands to the image
    image = image.addBands([NDVI, MNDWI, EVI, NDYI, PSRI]).float()

    return image


In [10]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)





In [21]:
s2_filtered = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(gfc)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.dayOfYear(start_doy, end_doy))
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
    .map(calculate_indices)
    .reduce(ee.Reducer.median())
    .select('NDVI_median', 'MNDWI_median',
            'EVI_median', 'NDYI_median', 'PSRI_median')
    .clip(gfc)
)

# s1_filtered = (
#     ee.ImageCollection("COPERNICUS/S1_GRD")
#     #.filterBounds(gfc)
#     #.filterDate(start_date, end_date)
#     #.filter(ee.Filter.dayOfYear(start_doy, end_doy))
#     .filter(ee.Filter.listContains('transmitterReceiverPolarisation',  ['VV', 'VH']))
#     .filter(ee.Filter.eq('instrumentMode', 'IW'))
#     .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
#     #.map(calculate_SAR_indices)
#     .select(["VV", "VH"])
#     #.reduce(ee.Reducer.median())
#     #.clip(gfc)
# )

naip_filtered = (
    naip.filterDate(start_date, end_date)
    #.filter(ee.Filter.dayOfYear(start_doy, end_doy))
    .filterBounds(gfc)
    #.sort('system:time_start', False)
    .mosaic()
)
naip_indices = (
    naip_filtered
    .addBands(naip_filtered.normalizedDifference(["G", "N"]).rename("ndwi"))
    .addBands(naip_filtered.normalizedDifference(["N", "R"]).rename("ndvi"))
)


print(s2_filtered.bandNames().getInfo())
print(naip_indices.bandNames().getInfo())
#(s1_filtered.first())

['NDVI_median', 'MNDWI_median', 'EVI_median', 'NDYI_median', 'PSRI_median']
['R', 'G', 'B', 'N', 'ndwi', 'ndvi']


In [22]:
#Map = geemap.Map(center=[48.5433, -118.8363], zoom=8)

# indices_to_visualize = ['NDVI_median', 'MNDWI_median', 'EVI_median', 'NDYI_median']
    
# for index_name in indices_to_visualize:
#     # Select index
#     index_image = s2_filtered.select(index_name)
        
#     # Add index layer to the map
#     Map.addLayer(index_image, {}, f'{index_name}')


Map.addLayer(naip_indices.select("ndwi"), {}, "NAIP Index")
#Map.addLayer(s1_filtered.select("DpRVIVV_median"), {}, "DpRVIVV")

In [27]:
naip_proj = naip_filtered.select("N").projection().getInfo()

In [37]:
features = geemap.fishnet(gfc, rows=50, cols=50)
Map.addLayer(features, {}, "Grids")

In [35]:
projection = s2_filtered.select('NDVI_median').projection().getInfo()

task = ee.batch.Export.image.toCloudStorage(
    image=naip_indices, 
    description='NYS_NAIP_Indices',
    #assetId= "projects/earthengineajs/assets/NYS_HUC_Indices",
    bucket=bucket_name,
    fileNamePrefix='NYS_NAIP_Indices_',
    crs="EPSG:4326",
    crsTransform=naip_proj["transform"],
    region=gfc.geometry(),
    maxPixels=1e12,
    scale=1,
    #formatOptions={'cloudOptimized': False},
    #fileFormat="GeoTIFF"
)
task.start()

EEException: The field "max_pixels" must have a value between 1 and 10000000000000 inclusive. If unspecified, the default value is 100000000.

In [86]:
# ee.batch.Task.status(task)

{'state': 'READY',
 'description': 'NYS_S2_Indices',
 'priority': 100,
 'creation_timestamp_ms': 1752858478657,
 'update_timestamp_ms': 1752858478657,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '3CSMXBX4OSKV2DE73DSWFOFQ',
 'name': 'projects/484202851264/operations/3CSMXBX4OSKV2DE73DSWFOFQ'}

In [87]:
ee.batch.Task.list()

[<Task 3CSMXBX4OSKV2DE73DSWFOFQ EXPORT_IMAGE: NYS_S2_Indices (READY)>,
 <Task SUPXL7PYIBCJ74MIGF5PE7HK EXPORT_IMAGE: NYS_S2_Indices (COMPLETED)>,
 <Task YGXORXKBUWNFPF6D53LJOGEQ EXPORT_IMAGE: NYS_S2_Indices (COMPLETED)>,
 <Task 6DUJUGAR5YEQQLVSWUXQBSXR EXPORT_IMAGE: NYS_S2_Indices (COMPLETED)>,
 <Task D3RUV2UED5JTESMRBD22ONRB EXPORT_IMAGE: NYS_S2_Indices (COMPLETED)>,
 <Task AMPKTOTYCRKKEKEEMCOVBKDX EXPORT_IMAGE: NYS_S2_Indices (COMPLETED)>,
 <Task KS5NA56H43LJBTOAFI3DL356 EXPORT_IMAGE: WA_10m_Indices (RUNNING)>,
 <Task 2BW5HK55LNWZQR7TX6N2PA7T EXPORT_IMAGE: WA_10m_Indices (FAILED)>,
 <Task OQ2UXXUPCA5ELJO4GS6RMSLA EXPORT_IMAGE: NYS_S2_Indices (COMPLETED)>]

In [32]:
from google.cloud import storage


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )



In [35]:
upload_blob(bucket_name=bucket_name, source_file_name="/Users/Anthony/Data and Analysis Local/NYS_Wetlands_GHG/Data/NWI/NY_Wetlands_6350.gpkg", destination_blob_name="NY_Wetlands_6350.gpkg")

File /Users/Anthony/Data and Analysis Local/NYS_Wetlands_GHG/Data/NWI/NY_6350.gpkg uploaded to test.gpkg.


## MODIS

In [ ]:
# MODIS

modis_pp = (
    ee.ImageCollection('MODIS/061/MOD17A3HGF')
    #.filterDate(start_date, end_date)
    #.filter(ee.Filter.dayOfYear(start_doy, end_doy))
    # Pre-filter to get less cloudy granules.
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .median()
    .clip(hlef_fc)
)

print(modis_pp)

## Hansen maps of forest cover

In [71]:
# Define the Hansen Global Forest Change dataset.
dataset = ee.Image('UMD/hansen/global_forest_change_2020_v1_8')

# Select the layers you are interested in.
# 'loss' indicates areas of forest loss.
forest_loss = dataset.select(['loss'])

# Clip the forest loss data to the AOI.
clipped_forest_loss = forest_loss.clip(gfc)

In [72]:
Map.addLayer(clipped_forest_loss)

In [73]:
projection_han = clipped_forest_loss.select('loss').projection().getInfo()

task_h = ee.batch.Export.image.toCloudStorage(
    image=clipped_forest_loss, 
    description='NYS_Hansen_ForLoss',
    bucket=bucket_name, 
    fileNamePrefix='GEE_Export',
    crs=projection_han['crs'],
    crsTransform=projection_han['transform'],
    region=gfc.geometry(),
    maxPixels=1e10,
    scale=30,
    fileFormat="GeoTIFF"
)
task_h.start()

In [86]:
ee.batch.Task.list()

[<Task 3C5V36EFOFUVKEYRBEBYF64E EXPORT_IMAGE: NYS_forest_loss_export (CANCEL_REQUESTED)>,
 <Task IDNCSNQ3GAYLRRKIQQEKSOQW EXPORT_IMAGE: NYS_forest_loss_export (COMPLETED)>,
 <Task OPH7CYSZ47LKFAVPDB2W574D EXPORT_IMAGE: NYS_Hansen_ForLoss (FAILED)>,
 <Task C6E4BQ6GZPL7OX5YBX2GB2Y2 EXPORT_IMAGE: NYS_S2_Indices (FAILED)>,
 <Task 3BCS6JPGBLGSOOSL7TD5LDMC EXPORT_IMAGE: NYS_S2_Indices (FAILED)>,
 <Task JS4L2HZX62HD5WYLWTJOEMA7 EXPORT_IMAGE: NYS_S2_Indices (FAILED)>,
 <Task DSPJ6DNVQX475OJGK3ARRE4Z EXPORT_IMAGE: NYS_S2_Indices (FAILED)>,
 <Task O4372R4UDAYJHW6WJNQTTN6T EXPORT_IMAGE: myExportImageTask (FAILED)>,
 <Task TPFCYE4P65NMIPMLS44IY3QI EXPORT_IMAGE: NYS_S2_Indices (FAILED)>]

In [81]:
# Get the URL for downloading the image.
export_params_hansen ={
    'image': clipped_forest_loss,
    'scale': 30,  # Scale in meters
    'folder': "GEE_Data_Export",
    'description': 'NYS_forest_loss_export',
    'region': gfc.geometry(),  # Area of interest
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e9
}

# If you want to export the data to your Google Drive, you can use:
# Export as GeoTIFF
task_hg = ee.batch.Export.image.toDrive(**export_params_hansen)
task_hg.start()

Landfire Vegetation 

In [ ]:
help(ee.Image.setMulti)

In [ ]:
# Define the Landfire dataset.
lf_dataset = (ee.Image('LANDFIRE/Vegetation/EVT/v1_4_0/CONUS')
              #.select("EVT")
              .clip(wa_hucs)
              #.get("EVT_class_names")
             #
             )



# Clip the landfire data to the AOI.
display('All metadata:', lf_dataset)

In [ ]:
Map = geemap.Map(center=[48.5433, -118.8363], zoom=8)

Map.addLayer(lf_dataset, {'min': 3001, 'max': 3968, 'palette': ['red', 'blue']}, "lf_wa")
Map

In [ ]:
import csv
import pandas as pd 
class_names = (lf_dataset.get('EVT_class_names')).getInfo()
class_values = (lf_dataset.get('EVT_class_values')).getInfo()

class_dict = {'names': class_names, 'values': class_values}

class_df = pd.DataFrame(class_dict)
class_df.to_csv("/Users/Anthony/OneDrive - UW/University of Washington/Data and Modeling/SOIL CARBON/All_WA/data/dataframes/WA_LandFire_Classes.csv")

In [ ]:
# Get the URL for downloading the image.
export_params_lf ={
    'image': lf_wa,
    'scale': 30,  # Scale in meters
    'description': 'WA_LandFire_2014',
    'folder': 'GEE Spatial Layers',
    'region': wa_hucs.geometry(),  # Area of interest
    'fileFormat': 'GeoTIFF',
    #'selectors': ['EVT_class_names'],
    'maxPixels': 1e12
}

# If you want to export the data to your Google Drive, you can use:
# Export as GeoTIFF
task_h = ee.batch.Export.image.toDrive(**export_params_lf)
task_h.start()

In [ ]:
geemap.ee_export_image_to_drive(
    lf_wa,
    folder="GEE_Spatial_Layers",
    #crs=crs,
    #crs_transform=crs_transform,
    region= wa_hucs.geometry(),
    scale = 30,
    maxPixels = 1e12
)

Climate Data from WorldClim

In [ ]:
# Define the WorldClim dataset
wclim_dataset = ee.Image('WORLDCLIM/V1/BIO')

# Select the layers you are interested in.
# 'loss' indicates areas of forest loss.
climate_data = (wclim_dataset.select(['bio01', 'bio04', 'bio12', 'bio15'])
               .multiply([0.1,0.01, 1.0, 1.0]))

wa_hucs_buff = wa_hucs.geometry().buffer(2000)

# Clip the forest loss data to the AOI.
clipped_climate_data = climate_data.clip(wa_hucs).toDouble()

In [ ]:
Map = geemap.Map(center=[48.5433, -118.8363], zoom=8)

clim_indices_to_visualize = ['bio01', 'bio04', 'bio12', 'bio15']
    
for index_name in clim_indices_to_visualize:
    # Select index
    index_image = clipped_climate_data.select(index_name)
        
    # Add index layer to the map
    Map.addLayer(index_image, {}, f'{index_name}')

Map.addLayer(clipped_climate_data)

In [ ]:
Map

In [ ]:
# Get the URL for downloading the image.
export_params_climate ={
    'image': clipped_climate_data,
    'scale': 927.67,  # Scale in meters
    'description': 'clipped_climate_data',
    'region': wa_hucs_buff,  # Area of interest
    'fileFormat': 'GeoTIFF',
    'maxPixels': 201565679
}

# If you want to export the data to your Google Drive, you can use:
# Export as GeoTIFF
task_h = ee.batch.Export.image.toDrive(**export_params_climate)
task_h.start()